# CO2 calculation bottom up methode; From power plant data to country aggregated CO2 intensity of electricity generation

In this script we calculated an CO2 emission factor per country out of hourly power plant generation and yearly published emission data on plant level.

The used method follows the idea to calculated a CI for specific power plants. In a second step an representative sample of power plants for a country is build and an CI for each technology and country is calculated.

# Script setup

In [126]:
import os
import logging

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from IPython.display import Image 

%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 10]

#helpers


# Data directory preparention

Create input, processed and output folders if they don't exist
If the paths are relative, the corresponding folders will be created inside the current working directory.

In [127]:
input_directory_path = os.path.join('input')
Bootom_up_methode_input_directory_path = os.path.join('input', 'Bootom_up_methode')
processed_directory_path = 'processed'
output_directory_path = os.path.join('output')

os.makedirs(input_directory_path, exist_ok=True)
os.makedirs(Bootom_up_methode_input_directory_path, exist_ok=True)
os.makedirs(processed_directory_path, exist_ok=True)
os.makedirs(output_directory_path, exist_ok=True)

# Data file preperation

The directory `input/Bootom_up_method` should contain all necessary raw data files.

Based on the developed method the following data sets needed:

1) Matcher for power plants in Entso e and EUTL Data for Europe
The matching was performed by a manual process (see description).

File - > Matching_Entso_EUTL_EU.csv

2) EU Emissions Data (EUTL)
Data are provided in the report section. See the following link:
https://ec.europa.eu/clima/policies/ets/registry_en#tab-0-1

For example: ->Documentation->Reports->Verified Emissions for 2019

File - > "Verified Emissions for 2019" -> verified_emissions_2018_en.xlsx -> converted to .csv

Another way to check the data is directly through the European Union Transaction Log:
https://ec.europa.eu/clima/ets/napMgt.do?languageCode=en

3) ENTSO e Data
Production per Unit from ENTSO-E
Original data: ENTSO-E Transparency Platform, Actual Generation per Generation Unit Available online: https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show (accessed on Apr 29, 2020).
Processed with the following script that converts the data to hourly resolution (private script from INATECH):
https://github.com/INATECH-CIG/entso-e_GenerationOutputPerUnit


File - > gen_data.csv -> hourly generation data per unit\

File - > unit_data.csv -> information about the generation units

4) Entso unit generation data from EMBER
The Entso e data has lot of gaps and need to be preprocessing (is done partly and the result is different than the one from Ember)

File - > entsoe_unit_gen_data.csv




In [128]:
# Checks if the the input directories are empty or not
# Checks all filenames in the input directory

if not os.listdir(Bootom_up_methode_input_directory_path) :
    print("The directory for the bootom up method is empty. Please provide the data to the directory as described in the instructions above.")


filenames = [os.path.join(Bootom_up_methode_input_directory_path, fn) for fn in os.listdir(Bootom_up_methode_input_directory_path)]

print(filenames)

['input\\Bootom_up_methode\\entsoe_unit_gen_data.csv', 'input\\Bootom_up_methode\\gen_data.csv', 'input\\Bootom_up_methode\\Matching_Entso_EUTL_EU.csv', 'input\\Bootom_up_methode\\Matching_Entso_EUTL_EU_2.csv', 'input\\Bootom_up_methode\\Matching_Entso_EUTL_LCPD.csv', 'input\\Bootom_up_methode\\unit_data.csv', 'input\\Bootom_up_methode\\verified_emissions_2018_en.csv', 'input\\Bootom_up_methode\\verified_emissions_2018_en.xlsx']


# Load data functions

In [43]:
def load_matching_data_EU(path, fn):
    """
    Matching List for EU power plants with Entso e identifier and the EUTL identifier.
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn), sep = ',', header = 0, index_col=0)

    return df

def load_EUTL_data(path, fn):
    """
    EU Emissions Data (EUTL)
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn),sep = ';',header = 13,encoding ='unicode_escape' )

    return df

def load_generation_data(path, fn):
    """
    Entso e gernation data per unit
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0,parse_dates=True)

    return df

def load_unit_data(path, fn):
    """
    Entso e generation unit information
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0)
    # set name for the index
    df.index.set_names('GenerationUnitEIC', inplace=True)

    return df

def load_unit_data_ember(path, fn):
    """
    Entso e unit generation from ember
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn), sep = ';', header = 0, encoding = 'unicode_escape')

    return df

# Load data sets

Load power plant generation data.

In [4]:
generation_per_unit = load_generation_data(Bootom_up_methode_input_directory_path, 'gen_data.csv')

Load power plant unit inforamtion (capacity, name, etc.)

In [129]:
generation_unit_info = load_unit_data(Bootom_up_methode_input_directory_path, 'unit_data.csv')

Load CO2_emissions data from EUTL dataset

In [6]:
EUTL_emissions = load_EUTL_data(Bootom_up_methode_input_directory_path, 'verified_emissions_2018_en.csv')

Load machting information for power plant data

In [ ]:
unit_matching_EU = load_matching_data_EU(Bootom_up_methode_input_directory_path, 'Matching_Entso_EUTL_EU.csv')

following data are not needed (testdata)

In [130]:
#generation_per_unit_ember = load_unit_data_ember(Bootom_up_methode_input_directory_path, 'entsoe_unit_gen_data.csv') 

#generation_per_unit_ember = generation_per_unit_ember[generation_per_unit_ember.Year == 2018]

#generation_per_unit_ember = generation_per_unit_ember.groupby(by = 'PowerSystemResourceName').sum()['Monthly Generation (GWh)']

In [131]:
#unit_matching_EU_2 = load_matching_data_EU(Bootom_up_methode_input_directory_path, 'Matching_Entso_EUTL_EU_2.csv')

# Yearly power generation per unit

Store the yearly generation per power plant to the power plant information data set.

In [132]:
generation_unit_info['yearly_generation_2018'] = generation_per_unit.sum()

following part not needed

vergleich unsere Jährliche Erzeugung mit denen, die wir von Ember bekommen haben (in 12, in Gfuellner Code) . Die unserscheidet sich. Was macht Ember noch mit den Daten?

In [133]:
#generation_unit_info[generation_unit_info.PowerSystemResourceName == 'Scheldelaan Exxonmobil']

Combining matching information to generation unit info

In [136]:
generation_unit_info_matcher = pd.merge(generation_unit_info, unit_matching_EU, on='PowerSystemResourceName', how='inner')

In [138]:
generation_unit_info_matcher.head()

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018,countrycode,EUTL_ID
0,10YAT-APG------L,APG CA,CTA,140.0,AT,Lau GuD,Fossil Gas,2.0,0.00,AT,86.0
1,10YAT-APG------L,APG CA,CTA,400.0,AT,Kraftwerk Timelkam GUD,Fossil Gas,2.0,685235.67,AT,149.0
2,10YAT-APG------L,APG CA,CTA,332.0,AT,KW Dürnrohr Block 2,Fossil Hard coal,2.0,745290.41,AT,94.0
3,10YAT-APG------L,APG CA,CTA,150.0,AT,KW Riedersbach 2 G2,Fossil Hard coal,2.0,0.00,AT,79.0
4,10Y1001A1001A796,Energinet CA,CTA,147.0,DK,Asnaesvaerket 2,Fossil Hard coal,1.0,219309.91,DK,48.0


In [ ]:
#not needed anymore
#generation_per_unit_ember = generation_per_unit_ember.groupby(by = 'PowerSystemResourceName').agg({'Monthly Generation (GWh)': 'sum',
                                                                       'MapCode':'first',
                                                                       'ProductionTypeName':'first',
                                                                       
                                                                         'Year': 'first',
                                                                         'Installed Capacity (MW)': 'first'
                                                                      })
#merged_ember = pd.merge(generation_per_unit_ember, unit_matching_EU_2, on='PowerSystemResourceName', how='inner')

# Def. function for connecting emissions and production data

In [99]:
def conv(x):
    # converts german grid operator areas string in german countrycode of ETS
    areas =['DE_TenneT_GER','DE_TransnetBW','DE_Amprion','DE_50HzT']
    if x in areas:
        x = 'DE' 
    return x

def connect_CO2(df, ETS):
    #df = df.drop(columns = 'index')
    df['countrycode']= df.countrycode.apply(lambda x: conv(x))
    # caring for characteristics of the dataset: EUTL-ID IS NOT UNIQUE - only countrywise.
    # removing power plants from the dataset which are not covered in ETS:
    miss_country = []
    for i in set(df.countrycode):
        if sum(i == ETS['REGISTRY_CODE']) == 0:
            print(i)
            miss_country.append(i)
            #removing this power plants from the list:
    df = df.query('countrycode not in @miss_country')
    
    # removing power plants where match could not be found:
    miss_match = []
    for j in df.PowerSystemResourceName:
        row = df[df.PowerSystemResourceName == j]
        if len(ETS.query('REGISTRY_CODE == @row.countrycode.iloc[0]')\
                  .query('INSTALLATION_IDENTIFIER == @row.EUTL_ID.iloc[0]')) == 0:
            print(j)
            miss_match.append(j)
            
    df = df.query('PowerSystemResourceName not in @miss_match')
    # apply matching:
    df['verified_emissions_18'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2018']\
                                    .iloc[0], axis = 1)
    df['verified_emissions_17'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2017']\
                                    .iloc[0], axis = 1)
    df['verified_emissions_16'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2016']\
                                    .iloc[0], axis = 1)
    df['verified_emissions_15'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2015']\
                                    .iloc[0], axis = 1)
    
    df['ETS_name'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['IDENTIFIER_IN_REG']\
                                    .iloc[0], axis = 1)
    return df

In [139]:
Powerplants_emission = connect_CO2(generation_unit_info_matcher, EUTL_emissions)

In [140]:
Powerplants_emission

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018,countrycode,EUTL_ID,verified_emissions_18,verified_emissions_17,verified_emissions_16,verified_emissions_15,ETS_name
0,10YAT-APG------L,APG CA,CTA,140.0,AT,Lau GuD,Fossil Gas,2.0,0.000,AT,86.0,6300,3825,3136,4097,Wienstrom KW Leopoldau Wien
1,10YAT-APG------L,APG CA,CTA,400.0,AT,Kraftwerk Timelkam GUD,Fossil Gas,2.0,685235.670,AT,149.0,267204,356994,213716,174692,Energie AG GuD Kraftwerk Timelkam
2,10YAT-APG------L,APG CA,CTA,332.0,AT,KW Dürnrohr Block 2,Fossil Hard coal,2.0,745290.410,AT,94.0,0,0,8,565110,Verbund KW Dürnrohr Zwentendorf
3,10YAT-APG------L,APG CA,CTA,150.0,AT,KW Riedersbach 2 G2,Fossil Hard coal,2.0,0.000,AT,79.0,6576,7775,203045,270681,KW Riedersbach
4,10Y1001A1001A796,Energinet CA,CTA,147.0,DK,Asnaesvaerket 2,Fossil Hard coal,1.0,219309.910,DK,48.0,878395,1048994,952588,695549,Asnæsværket
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-3,Fossil Hard coal,0.0,236678.180,GB,169.0,540497,1014334,2098992,4748504,Eggborough Operator Account
797,10YGB----------A,National Grid CA,CTA,260.0,GB,DEEP-1,Fossil Gas,0.0,335757.515,GB,187.0,145298,289810,248853,141965,Operator Account
798,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-2,Fossil Hard coal,0.0,15438.130,GB,169.0,540497,1014334,2098992,4748504,Eggborough Operator Account
799,10YGB----------A,National Grid CA,CTA,20.0,GB,WBUGT-4,Fossil Gas,0.0,81.230,GB,145.0,1679509,1766100,1165058,7724207,West Burton A


In [141]:
def calc_CI(df):
    # calculates the carbon intensity of sites. Due to non-unique labelings of the power plants across different
    # countries it needs care taken about it.
    
    df['verified_emissions_18']=df['verified_emissions_18'].apply(float)
    
    sites = pd.DataFrame(df.groupby(['countrycode','EUTL_ID']).mean()['verified_emissions_18']\
                       / df.groupby(['countrycode','EUTL_ID']).sum()['yearly_generation_2018'])\
                        .reset_index()
    df['carbon_intensity'] = df.apply(lambda x: sites.query('countrycode == @x.countrycode')\
                            .query('EUTL_ID == @x.EUTL_ID')[0].iloc[0],axis = 1)
    
    return df

In [142]:
Powerplants_emission_EU_CI = calc_CI(Powerplants_emission)

In [151]:
Powerplants_emission_EU_CI.head()

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018,countrycode,EUTL_ID,verified_emissions_18,verified_emissions_17,verified_emissions_16,verified_emissions_15,ETS_name,carbon_intensity
0,10YAT-APG------L,APG CA,CTA,140.0,AT,Lau GuD,Fossil Gas,2.0,0.00,AT,86.0,6300.0,3825,3136,4097,Wienstrom KW Leopoldau Wien,inf
1,10YAT-APG------L,APG CA,CTA,400.0,AT,Kraftwerk Timelkam GUD,Fossil Gas,2.0,685235.67,AT,149.0,267204.0,356994,213716,174692,Energie AG GuD Kraftwerk Timelkam,0.389945
2,10YAT-APG------L,APG CA,CTA,332.0,AT,KW Dürnrohr Block 2,Fossil Hard coal,2.0,745290.41,AT,94.0,0.0,0,8,565110,Verbund KW Dürnrohr Zwentendorf,0.000000
3,10YAT-APG------L,APG CA,CTA,150.0,AT,KW Riedersbach 2 G2,Fossil Hard coal,2.0,0.00,AT,79.0,6576.0,7775,203045,270681,KW Riedersbach,inf
4,10Y1001A1001A796,Energinet CA,CTA,147.0,DK,Asnaesvaerket 2,Fossil Hard coal,1.0,219309.91,DK,48.0,878395.0,1048994,952588,695549,Asnæsværket,1.140746


# Validation of results

In [190]:
# covert technology
tech = ['Fossil Gas', 'Fossil Hard coal',
       'Fossil Brown coal/Lignite']

Powerplants_emission_EU_CI = Powerplants_emission_EU_CI.query('ProductionTypeName in @tech')

# not covert : 'Fossil Oil shale', 'Fossil Oil', 'Fossil Coal-derived gas','Fossil Peat'

coverage = len(Powerplants_emission_EU_CI) / len(generation_unit_info.query('ProductionTypeName in @tech'))
print("We were able to allocate " + str(coverage*100) + " % of the power plants an emission value.")

We were able to allocate 89.14285714285714 % of the power plants an emission value.


Since not all emission factors are correctly calculated we limited the list of calculated emissions per power plant is checked by a plausibility check.

thresholds for plausibility check - until [g/kWh]: 
lignite_high = 1750
lignite_low = 800

hardcoal_high = 1350
hardcoal_low = 700

gas_high = 550
gas_low = 350

Numbers taken from:
https://www.gegenwind-saarland.de/Materialien/Energiewende/071031--VdI---CO2-Emissionen%20der%20Stromerzeugung_01.pdf

In [191]:
# emissions space per technology in [g/kWh] 

emission_space = {"Fossil Brown coal/Lignite": (1750,800),
                  "Fossil Hard coal": (1350,700),
                  "Fossil Gas": (550,350)}

In [192]:
# function for CI check
def check_CI (row):
    if row.carbon_intensity > emission_space[row.ProductionTypeName][1] and row.carbon_intensity < emission_space[row.ProductionTypeName][0]:
        return True
    else:
        return False  

In [200]:
# convert emissions to [g/kWh]
Powerplants_emission_EU_CI.carbon_intensity = Powerplants_emission_EU_CI.carbon_intensity * 1000

In [201]:
Powerplants_emission_EU_CI['CI_validation'] = Powerplants_emission_EU_CI.apply(check_CI, axis=1)

In [204]:
Powerplants_emission_EU_CI = Powerplants_emission_EU_CI[Powerplants_emission_EU_CI.CI_validation == True]

In [208]:
coverage_val = len(Powerplants_emission_EU_CI) / len(generation_unit_info.query('ProductionTypeName in @tech'))
print("After validation we were able to allocate " + str(coverage_val*100) + " % of the power plants an emission value.")

After validation we were able to allocate 55.2 % of the power plants an emission value.


reasons why the CI is not valid:

1.) Match is not correct - then one more iteration is necessary

2.) Data is wrong or incomplete

3.) Match is correct, but it's a highly aggregated facility such like a steel manufacturing factory, where both process-CO2 AND the emissions from the power plant running the process accounts into the number given in the dataset.

4.) the facility contains more smaller power plants which are not covered by ENTSO-E (<100MW) but nevertheless
produce emissions which need to be paid for.

5.) if the number of the carbon-intensity is negative, then the dataset is explained as " -1 = blank (No allocation has been made / No Emissions have been Verified)"

# Carbon intensity per technology and country 

In [215]:
def calc_CI_tech_country(df):
    # calculates the average carbon intensity per technology in a country for plausible results defined above.
    # df is a dataframe which contains all sites with plausible carbon intensities of whole Europe.
    CI_tech_country = df.groupby(['countrycode','EUTL_ID','ProductionTypeName'])\
                                     .mean()['verified_emissions_18']\
                                     .groupby(['countrycode','ProductionTypeName']).sum()\
                                     /\
                      df.groupby('countrycode')['yearly_generation_2018'].sum()
    print(CI_tech_country)
    return CI_tech_country

In [220]:
# form the gourp out of the different power plant per country
CI_tech_country = Powerplants_emission_EU_CI.groupby(['countrycode','ProductionTypeName'])\
                                            .mean()['carbon_intensity']

In [246]:
CI_tech_country[('DE','Fossil Brown coal/Lignite')]

1151.2639041959112

In [247]:
# Plot CI per country for the technologies (drei Plots)

In [212]:
Powerplants_emission_EU_CI.columns

Index(['AreaCode', 'AreaName', 'AreaTypeCode', 'InstalledGenCapacity',
       'MapCode', 'PowerSystemResourceName', 'ProductionTypeName',
       'duplicate_count', 'yearly_generation_2018', 'countrycode', 'EUTL_ID',
       'verified_emissions_18', 'verified_emissions_17',
       'verified_emissions_16', 'verified_emissions_15', 'ETS_name',
       'carbon_intensity', 'drop', 'CI_validation'],
      dtype='object')

In [ ]:
# Teste wie viel Energie decken wir mit dieser Methode ab?
# Dafür welche Daten verwenden (Problem Gross vs Net)
# Gibt es von Eurostat eine Net Version (ich glaube ja pro Monat)
# Dann Plot pro Land wie viel Dekcen wir ab (0 bis 1)



In [ ]:
# wie könnten wir ein representatives sample bilder. Das soll uns dann die CI ausrechnen

# Wie machen wir den Vergleich zur anderen Methode?

In [ ]:
# Jetzt können wir dann CI pro technologie bestimmen. 